# Model Context Protocol (MCP) Demo with Anthropic Messages API

This notebook demonstrates the basic usage of Anthropic's Model Context Protocol (MCP) for connecting AI models to external data sources and tools.

## What is MCP?

MCP (Model Context Protocol) is an open standard by Anthropic that standardizes how AI models connect to external tools and data sources. Think of it as a "USB-C port for AI applications" - it provides a unified way to connect to different services instead of building custom integrations for each one.

## Key Benefits:
- **Standardized**: One protocol for all external connections
- **Secure**: Built-in authentication and security practices
- **Flexible**: Works with local and remote servers
- **Open**: Not limited to Anthropic - adopted by OpenAI and others

## Required Environment Variables

```
ANTHROPIC_API_KEY    # Your Anthropic API key (required)
```

Please refer to the [README](README.md) for instructions on setting up environment variables.

## Setup

First, let's install the required packages and set up our environment.

In [ ]:
# If running standalone without the project setup:
# pip install anthropic python-dotenv requests mcp nest-asyncio

In [1]:
import os
import json
import requests
import asyncio
from anthropic import Anthropic
from dotenv import load_dotenv
from mcp.server.fastmcp import FastMCP
import nest_asyncio

# Enable nested asyncio for Jupyter
nest_asyncio.apply()

# Load environment variables
load_dotenv(override=True)

# Initialize Anthropic client
client = Anthropic(
    api_key=os.getenv('ANTHROPIC_API_KEY')  # Make sure to set your API key in .env file
)

## Basic MCP Configuration

Let's set up a basic MCP server configuration. For this demo, we'll create a mock MCP server endpoint that simulates a weather service.

In [2]:
# MCP Server configuration
# In a real scenario, this would be a running MCP server
# For demo purposes, we'll simulate the MCP interaction

mcp_server_config = {
    "type": "url",
    "url": "https://api.openweathermap.org/data/2.5",  # Using OpenWeatherMap as example
    "name": "weather-api",
    "authorization_token": "demo_token",  # In real usage, get proper API key
    "tool_configuration": {
        "enabled": True,
        "allowed_tools": ["get_weather"]
    }
}

print("MCP Server Configuration:")
print(json.dumps(mcp_server_config, indent=2))

MCP Server Configuration:
{
  "type": "url",
  "url": "https://api.openweathermap.org/data/2.5",
  "name": "weather-api",
  "authorization_token": "demo_token",
  "tool_configuration": {
    "enabled": true,
    "allowed_tools": [
      "get_weather"
    ]
  }
}


## MCP Demo with External API

Since setting up a full MCP server requires additional infrastructure, we'll demonstrate the MCP concept using Anthropic's Messages API with the MCP connector feature.

**Important:** The MCP connector feature requires the beta header `"anthropic-beta": "mcp-client-2025-04-04"`

## Real MCP Server Implementation

Let's create an actual MCP server for weather data using the Open-Meteo API (free, no API key required). This will demonstrate how MCP servers expose tools that AI models can use.

In [3]:
# Create a simple MCP server for weather data
# Using Open-Meteo API (free, no API key required)

mcp = FastMCP("weather-server")

@mcp.tool()
async def get_weather(location: str = "Singapore") -> dict:
    """
    Get current weather for a location.
    Defaults to Singapore if no location specified.

    Args:
        location: City name (e.g., "Singapore", "Tokyo", "London")

    Returns:
        Current weather data including temperature, conditions, and more
    """
    # Coordinates for common cities (for demo simplicity)
    city_coords = {
        "singapore": {"lat": 1.3521, "lon": 103.8198},
        "tokyo": {"lat": 35.6762, "lon": 139.6503},
        "london": {"lat": 51.5074, "lon": -0.1278},
        "new york": {"lat": 40.7128, "lon": -74.0060},
        "sydney": {"lat": -33.8688, "lon": 151.2093}
    }

    # Get coordinates for the city
    coords = city_coords.get(location.lower(), city_coords["singapore"])

    # Fetch weather from Open-Meteo API (free, no key required)
    url = f"https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": coords["lat"],
        "longitude": coords["lon"],
        "current": "temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m",
        "timezone": "auto"
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()

        # Weather code descriptions
        weather_codes = {
            0: "Clear sky",
            1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
            45: "Foggy", 48: "Depositing rime fog",
            51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
            61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
            71: "Slight snow", 73: "Moderate snow", 75: "Heavy snow",
            77: "Snow grains",
            80: "Slight rain showers", 81: "Moderate rain showers", 82: "Violent rain showers",
            85: "Slight snow showers", 86: "Heavy snow showers",
            95: "Thunderstorm", 96: "Thunderstorm with slight hail", 99: "Thunderstorm with heavy hail"
        }

        current = data["current"]
        weather_code = current.get("weather_code", 0)

        return {
            "location": location.title(),
            "temperature": f"{current['temperature_2m']}°C",
            "condition": weather_codes.get(weather_code, "Unknown"),
            "humidity": f"{current['relative_humidity_2m']}%",
            "wind_speed": f"{current['wind_speed_10m']} km/h",
            "source": "Open-Meteo API via MCP",
            "timestamp": current["time"]
        }
    except Exception as e:
        return {"error": f"Failed to fetch weather: {str(e)}"}

## Testing the MCP Weather Tool

Let's test our MCP weather tool by fetching real weather data for Singapore:

In [4]:
# Test the MCP tool directly - fetch real Singapore weather
print("🌤️ Fetching real weather data for Singapore via MCP...\n")
singapore_weather = asyncio.run(get_weather("Singapore"))
print("Singapore Weather (Real-time data):")
print(json.dumps(singapore_weather, indent=2))

print("\n" + "="*50 + "\n")

# Also test for another city
print("🌍 Fetching weather for Tokyo...\n")
tokyo_weather = asyncio.run(get_weather("Tokyo"))
print("Tokyo Weather (Real-time data):")
print(json.dumps(tokyo_weather, indent=2))

🌤️ Fetching real weather data for Singapore via MCP...

Singapore Weather (Real-time data):
{
  "location": "Singapore",
  "temperature": "29.0\u00b0C",
  "condition": "Slight rain showers",
  "humidity": "75%",
  "wind_speed": "10.1 km/h",
  "source": "Open-Meteo API via MCP",
  "timestamp": "2025-09-18T16:15"
}


🌍 Fetching weather for Tokyo...

Tokyo Weather (Real-time data):
{
  "location": "Tokyo",
  "temperature": "28.5\u00b0C",
  "condition": "Mainly clear",
  "humidity": "64%",
  "wind_speed": "4.7 km/h",
  "source": "Open-Meteo API via MCP",
  "timestamp": "2025-09-18T17:15"
}


## Using Anthropic Messages API with MCP Concept

Now let's demonstrate how this would work with Anthropic's Messages API. We'll create a conversation where Claude needs to fetch external data to answer a question.

In [5]:
# Define tools for Claude to use (these match our MCP server tools)
tools = [
    {
        "name": "get_weather",
        "description": "Get current weather information for a location. Defaults to Singapore if no location specified.",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city to get weather for (e.g., Singapore, Tokyo, London)"
                }
            },
            "required": []
        }
    }
]

print("Available MCP tools:")
for tool in tools:
    print(f"- {tool['name']}: {tool['description']}")

Available MCP tools:
- get_weather: Get current weather information for a location. Defaults to Singapore if no location specified.


In [6]:
# Tool execution function (connecting to our MCP server)
def execute_tool(tool_name, tool_input):
    """
    Execute a tool call - this connects to our MCP server
    """
    if tool_name == "get_weather":
        # Call our actual MCP weather tool
        location = tool_input.get("location", "Singapore")
        result = asyncio.run(get_weather(location))
        return result
    else:
        return {"error": f"Unknown tool: {tool_name}"}

# Test tool execution with real data
print("🌤️ Testing MCP tool execution for Singapore:")
test_singapore = execute_tool("get_weather", {"location": "Singapore"})
print(json.dumps(test_singapore, indent=2))

print("\n" + "="*50 + "\n")

print("🌏 Testing MCP tool execution for Tokyo:")
test_tokyo = execute_tool("get_weather", {"location": "Tokyo"})
print(json.dumps(test_tokyo, indent=2))

🌤️ Testing MCP tool execution for Singapore:
{
  "location": "Singapore",
  "temperature": "28.9\u00b0C",
  "condition": "Slight rain showers",
  "humidity": "76%",
  "wind_speed": "9.8 km/h",
  "source": "Open-Meteo API via MCP",
  "timestamp": "2025-09-18T16:30"
}


🌏 Testing MCP tool execution for Tokyo:
{
  "location": "Tokyo",
  "temperature": "28.1\u00b0C",
  "condition": "Mainly clear",
  "humidity": "66%",
  "wind_speed": "4.7 km/h",
  "source": "Open-Meteo API via MCP",
  "timestamp": "2025-09-18T17:30"
}


## Complete MCP-Style Conversation Demo

Let's create a complete example showing how Claude would use MCP-style tools to answer user questions.

In [7]:
# Simulate a conversation with tool usage
def run_mcp_conversation(user_message):
    print(f"User: {user_message}\n")

    # First API call to Claude
    response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=1000,
        tools=tools,
        messages=[
            {
                "role": "user",
                "content": user_message
            }
        ]
    )

    print("Claude's response:")

    # Check if Claude wants to use tools
    if response.stop_reason == "tool_use":
        print("Claude is calling MCP tools to get real weather data...\n")

        # Extract tool calls
        tool_results = []
        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": response.content}
        ]

        for content_block in response.content:
            if content_block.type == "tool_use":
                tool_name = content_block.name
                tool_input = content_block.input
                tool_id = content_block.id

                print(f"🔧 Using MCP tool: {tool_name}")
                print(f"📥 Input: {tool_input}")

                # Execute the tool (real MCP call)
                tool_result = execute_tool(tool_name, tool_input)

                print(f"📤 Real data from MCP:")
                print(json.dumps(tool_result, indent=4))
                print()

                # Add tool result to conversation
                tool_results.append({
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_id,
                            "content": json.dumps(tool_result)
                        }
                    ]
                })

        # Add tool results to messages
        messages.extend(tool_results)

        # Get final response from Claude
        final_response = client.messages.create(
            model="claude-3-5-haiku-20241022",
            max_tokens=1000,
            tools=tools,
            messages=messages
        )

        print("Claude's final answer (based on real data):")
        for content_block in final_response.content:
            if content_block.type == "text":
                print(content_block.text)

    else:
        # Claude answered without tools
        for content_block in response.content:
            if content_block.type == "text":
                print(content_block.text)

    print("\n" + "="*50 + "\n")

## MCP-Style Conversation Demo with Real Weather API

Now let's see how Claude uses our MCP weather tool to answer questions with real-time data:

In [8]:
# Demo 1: Singapore weather query (real-time data!)
print("🇸🇬 SINGAPORE WEATHER DEMO\n")
print("This will fetch REAL weather data for Singapore using MCP...\n")
run_mcp_conversation("What's the current weather in Singapore? Please provide all details.")

🇸🇬 SINGAPORE WEATHER DEMO

This will fetch REAL weather data for Singapore using MCP...

User: What's the current weather in Singapore? Please provide all details.



[09/18/25 16:58:50] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=434247;file:///mnt/sda/yifen/myenv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=863759;file:///mnt/sda/yifen/myenv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Claude's response:
Claude is calling MCP tools to get real weather data...

🔧 Using MCP tool: get_weather
📥 Input: {'location': 'Singapore'}
📤 Real data from MCP:
{
    "location": "Singapore",
    "temperature": "28.7\u00b0C",
    "condition": "Slight rain showers",
    "humidity": "77%",
    "wind_speed": "9.2 km/h",
    "source": "Open-Meteo API via MCP",
    "timestamp": "2025-09-18T16:45"
}



[09/18/25 16:58:54] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=788509;file:///mnt/sda/yifen/myenv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=514405;file:///mnt/sda/yifen/myenv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Claude's final answer (based on real data):
Here are the detailed weather conditions for Singapore:

🌡️ Temperature: 28.7°C 
☁️ Current Condition: Slight rain showers
💧 Humidity: 77%
💨 Wind Speed: 9.2 km/h

The weather seems to be typical for Singapore - warm with some light rain. The temperature is quite comfortable at around 28.7°C, which is characteristic of Singapore's tropical climate. The slight rain showers and high humidity are also common in this region. The wind is blowing gently at 9.2 km/h, which might provide a bit of cooling effect.

Would you like me to elaborate on any of these weather details?




In [9]:
# Demo 2: Weather comparison
print("🌏 WEATHER COMPARISON DEMO\n")
run_mcp_conversation("Can you compare the current weather between Singapore and Tokyo? Which city is warmer?")

🌏 WEATHER COMPARISON DEMO

User: Can you compare the current weather between Singapore and Tokyo? Which city is warmer?



[09/18/25 17:01:07] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=505621;file:///mnt/sda/yifen/myenv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=298383;file:///mnt/sda/yifen/myenv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Claude's response:
Claude is calling MCP tools to get real weather data...

🔧 Using MCP tool: get_weather
📥 Input: {'location': 'Singapore'}
📤 Real data from MCP:
{
    "location": "Singapore",
    "temperature": "28.6\u00b0C",
    "condition": "Overcast",
    "humidity": "77%",
    "wind_speed": "8.9 km/h",
    "source": "Open-Meteo API via MCP",
    "timestamp": "2025-09-18T17:00"
}

🔧 Using MCP tool: get_weather
📥 Input: {'location': 'Tokyo'}
📤 Real data from MCP:
{
    "location": "Tokyo",
    "temperature": "27.5\u00b0C",
    "condition": "Mainly clear",
    "humidity": "69%",
    "wind_speed": "4.4 km/h",
    "source": "Open-Meteo API via MCP",
    "timestamp": "2025-09-18T18:00"
}



[09/18/25 17:01:12] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=251241;file:///mnt/sda/yifen/myenv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=618402;file:///mnt/sda/yifen/myenv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Claude's final answer (based on real data):
Let me break down the weather comparison for you:

Singapore:
- Temperature: 28.6°C
- Condition: Overcast
- Humidity: 77%

Tokyo:
- Temperature: 27.5°C
- Condition: Mainly clear
- Humidity: 69%

Singapore is slightly warmer, with a temperature of 28.6°C compared to Tokyo's 27.5°C. The difference is about 1.1°C. 

Both cities are experiencing warm weather, which is typical for their tropical and subtropical climates. Singapore is slightly warmer and more humid, with an overcast sky, while Tokyo is a bit cooler with mainly clear skies.

So to directly answer your question: Singapore is the warmer city at the moment.




In [10]:
# Demo 3: Travel planning with real weather data
print("✈️ TRAVEL PLANNING DEMO\n")
run_mcp_conversation("I'm planning to travel from Singapore to London next week. How's the weather difference between these two cities right now?")

✈️ TRAVEL PLANNING DEMO

User: I'm planning to travel from Singapore to London next week. How's the weather difference between these two cities right now?



[09/18/25 17:02:13] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=249908;file:///mnt/sda/yifen/myenv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=479354;file:///mnt/sda/yifen/myenv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Claude's response:
Claude is calling MCP tools to get real weather data...

🔧 Using MCP tool: get_weather
📥 Input: {'location': 'Singapore'}
📤 Real data from MCP:
{
    "location": "Singapore",
    "temperature": "28.6\u00b0C",
    "condition": "Overcast",
    "humidity": "77%",
    "wind_speed": "8.9 km/h",
    "source": "Open-Meteo API via MCP",
    "timestamp": "2025-09-18T17:00"
}

🔧 Using MCP tool: get_weather
📥 Input: {'location': 'London'}
📤 Real data from MCP:
{
    "location": "London",
    "temperature": "19.8\u00b0C",
    "condition": "Partly cloudy",
    "humidity": "78%",
    "wind_speed": "18.2 km/h",
    "source": "Open-Meteo API via MCP",
    "timestamp": "2025-09-18T10:00"
}



[09/18/25 17:02:21] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=336420;file:///mnt/sda/yifen/myenv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=930211;file:///mnt/sda/yifen/myenv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Claude's final answer (based on real data):
Let me break down the weather differences for you:

Singapore:
- Temperature: 28.6°C (quite warm)
- Condition: Overcast
- Humidity: 77%
- Wind Speed: 8.9 km/h

London:
- Temperature: 19.8°C (cooler)
- Condition: Partly cloudy
- Humidity: 78%
- Wind Speed: 18.2 km/h

Key differences:
1. Temperature: There's a significant temperature difference of about 8.8°C. Singapore is much warmer, which is typical of its tropical climate.
2. Conditions: Singapore is overcast, while London is partly cloudy.
3. Wind: London has stronger winds at 18.2 km/h compared to Singapore's 8.9 km/h.
4. Humidity: Both cities have similar humidity levels around 77-78%.

Recommendations:
- Pack layers for London, as the temperature is cooler
- Bring a light jacket or sweater
- Consider a wind-resistant jacket due to the higher wind speed
- The weather is mild, so comfortable clothing should suffice

The weather difference highlights the contrast between Singapore's tropic